In [1]:
from aie_kernel_builder import AIEKernelBuilder
import numpy as np

In [2]:
%%kernel
#include "ipulog.h"

void passthrough(uint8_t* a, uint8_t* c, uint8_t* stdout) {

    IPULogger log(stdout, 256);
    ::aie::vector<uint8_t, 32> ai, bi, ci;

    log.write("Starting Kernel Execution!\n");

    aie::tile tile=aie::tile::current();
    uint64_t Tstart = tile.cycles(); //cycle counter of the AI Engine tile

    for(int i=0; i<4096; i++) {
        c[i] = a[i];
    }

    uint64_t Tend = tile.cycles(); //cycle counter of the AI Engine tile
    uint64_t cycles = Tend - Tstart;

    log.write("Completed executing. cycles=%u\n", cycles);
}

In [3]:
a = np.random.randint(0, 256, size=4096, dtype=np.uint8)
c = np.zeros(4096, dtype=np.uint8)

passthrough.c.array = c
passthrough_test = AIEKernelBuilder(passthrough, a, c)

In [4]:
ans = passthrough_test.run(cache=False)

Using cached passthrough kernel object file...
Building the xclbin...
Successfully Building Application... AIEKernelBuilder.xclbin & AIEKernelBuilder.seq delivered
Build completed, running...


Starting Kernel Execution!
Completed executing. cycles=32776
~~~~~~~~~~~~~


In [5]:
ans

PynqBuffer([195, 137,   5, ..., 141,  94, 112], dtype=uint8)